In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Dataset = '/content/drive/MyDrive/BRAIN MRI DATASET'

In [ ]:
#Importing libraries
%matplotlib inline
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, random, shutil
import tensorflow as tf
import seaborn
from glob import glob
from tensorflow import keras
from tensorflow.keras import preprocessing, layers
from tensorflow.keras.callbacks import EarlyStopping
from keras_preprocessing import image
import PIL
import cv2
from keras.constraints import maxnorm
from keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D, Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator

from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split
from cv2 import cv2
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt

from keras.layers import Dense, Dropout, Flatten, Input, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.models import Model
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from numpy import array 
from keras import regularizers
from keras import optimizers
from keras.models import load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import pathlib
from sklearn.model_selection import train_test_split
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True



In [ ]:
def getTotalImages():

  total_images = 0
  for c in ['G','M','N','P']:
    total_images += len(os.listdir(os.path.join(Dataset, c)))
  # print('Total:', total_images)
  return total_images

In [ ]:
def defineTumourLabels():
  data_dir = ""
  data_dir = pathlib.Path(Dataset)
  Tumor_images_dict= {}
  Tumor_labels_dict = {}
  Tumor_images_dict= {
      'G': list(data_dir.glob('G/*')),
      'M': list(data_dir.glob('M/*')),
      'N': list(data_dir.glob('N/*')),  
      'P': list(data_dir.glob('P/*'))
  }
  Tumor_labels_dict = {
      'G':0,
      'M': 1,
      'N': 2,
      'P': 3,
  }
  return Tumor_images_dict,Tumor_labels_dict

In [ ]:
def preProcess(Tumor_images_dict,Tumor_labels_dict):
  print("\n Enter the Dataset Size")
  Dataset_Size = input()
  Dataset_Size = int(Dataset_Size)
  X, y = [],[]

  for Tumor_name, images in Tumor_images_dict.items():
      for image in images[0:Dataset_Size]:
          img= cv2.imread(str(image))
          im_color = cv2.applyColorMap(img, cv2.COLORMAP_JET)
          resized_img = cv2.resize(im_color,(180,180))
          X.append(resized_img)
          y.append(Tumor_labels_dict[Tumor_name])
  return X,y     

In [ ]:
def split(X,y):
  X =  np.array(X)
  y = np.array(y)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0) #input 2
  X_train_scaled = X_train / 255
  X_test_scaled = X_test / 255
  return X_train_scaled,X_test_scaled,y_train,y_test

In [ ]:
def struct_model():
    num_classes = 4;
    model = Sequential([
    #input 3             
    layers.Conv2D(275,(3,3), padding='same', activation='relu', input_shape=(180,180,3)),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),

    layers.Conv2D(250,(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),

    layers.Conv2D(225,(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    
    layers.Flatten(),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(50, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(num_classes,activation='softmax')  
])
    return model

In [ ]:
def Validation_Process(model,X_test_scaled, y_test):
  show_validation_accuracy(model,X_test_scaled, y_test)
  cm = show_classification_report(model,X_test_scaled, y_test)
  show_confusion_matrix(cm)

In [ ]:
def train(model,X_train_scaled, y_train,X_test_scaled, y_test):
  model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
  #input 4
  early_stop = EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience=10, restore_best_weights=True)
  history = model.fit(X_train_scaled, y_train, epochs=20, validation_data= (X_test_scaled, y_test))
  return history

In [ ]:
def print_model_summary(model):
  model.summary()

In [ ]:
def show_validation_accuracy(model,X_test_scaled, y_test):
  model.evaluate(X_test_scaled, y_test)

In [ ]:
def show_classification_report(model,X_test_scaled, y_test):
  from sklearn.metrics import confusion_matrix , classification_report
  import numpy as np
  y_predict =  model.predict(X_test_scaled)
  y_predict_classes = [np.argmax(element) for element in y_predict]
  print("Classification Report: \n", classification_report(y_test, y_predict_classes))
  cm = tf.math.confusion_matrix(labels=y_test,predictions=y_predict_classes)
  return cm

In [ ]:
def show_confusion_matrix(cm):
  import seaborn as sn
  plt.figure(figsize = (10,8))
  sn.heatmap(cm, annot=True, fmt='d')
  plt.xlabel('Predicted')
  plt.ylabel('Truth')

In [ ]:
def Learning_Process():
  print("Learning...")
  total_images = getTotalImages()
  print(total_images)
  Tumor_images_dict,Tumor_labels_dict = defineTumourLabels()
  print("Preprocessing...")
  X,y = preProcess(Tumor_images_dict,Tumor_labels_dict)
  print("Preprocessing Done!")
  X_train_scaled,X_test_scaled, y_train, y_test = split(X,y)
  model = struct_model()
  print_model_summary(model)
  history = train(model,X_train_scaled, y_train,X_test_scaled, y_test)
  print("Learned!")
  print("Validation on process...")
  Validation_Process(model, X_test_scaled, y_test)


In [ ]:
def names(number):
    if number==0:
        return 'Diagnosed with Glioma Tumor'
    elif number==1:
        return 'Diagnosed with Meningioma Tumor'
    elif number==2:
        return 'Diagnosed with No Tumor'
    else:
        return 'Diagnosed with Pituitary Tumor'

In [ ]:
def Recognition_Process():
  from matplotlib.pyplot import imshow
  print("Recongnition...")
  print("Enter image path for recognition:")
  imgpath = input()
  img = Image.open(imgpath)
  x = np.array(img.resize((180,180)))
  x = x.reshape(1,180,180,3)
  res = model.predict_on_batch(x)
  classification = np.where(res == np.amax(res))[1][0]
  imshow(img)
  print(str(res[0][classification]*100) + '% Confidence This Is ' + names(classification))

In [ ]:
def main():
  print("Welcome to User Interaction Menu")
  print("--------------------------------")
  var = 2
  while(var==1 or var==2):
    print("1.Learning Phase")
    print("2.Recognition Phase")
    print("3.End Program")
    print("Enter an option:")
    var = input()
    var = int(var)
    if var == 1:
      print("Starting learning...")
      Learning_Process()
    elif var == 2:
      Recognition_Process()

In [ ]:
if __name__=="__main__":
    main()

Welcome to User Interaction Menu
--------------------------------
1.Learning Phase
2.Recognition Phase
3.End Program
Enter an option:
1
Starting learning...
Learning...
3319
Preprocessing...

 Enter the Dataset Size
50
Preprocessing Done!
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 180, 180, 275)     7700      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 275)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 90, 90, 275)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 250)       619000    
                                                                 
 max_pooling2d_

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


5/5 [==============================] - 16s 493ms/step - loss: 2.1210 - accuracy: 0.1375 - val_loss: 1.3883 - val_accuracy: 0.2750
Epoch 2/20
5/5 [==============================] - 1s 306ms/step - loss: 1.3836 - accuracy: 0.2688 - val_loss: 1.3391 - val_accuracy: 0.4000
Epoch 3/20
5/5 [==============================] - 1s 304ms/step - loss: 1.2399 - accuracy: 0.4187 - val_loss: 1.0020 - val_accuracy: 0.7750
Epoch 4/20
5/5 [==============================] - 1s 305ms/step - loss: 0.9249 - accuracy: 0.5875 - val_loss: 0.6488 - val_accuracy: 0.7500
Epoch 5/20
5/5 [==============================] - 1s 302ms/step - loss: 0.8596 - accuracy: 0.7125 - val_loss: 0.6826 - val_accuracy: 0.7000
Epoch 6/20
5/5 [==============================] - 1s 304ms/step - loss: 0.7390 - accuracy: 0.7125 - val_loss: 0.6585 - val_accuracy: 0.7500
Epoch 7/20
5/5 [==============================] - 1s 304ms/step - loss: 0.8145 - accuracy: 0.7250 - val_loss: 0.5695 - val_accuracy: 0.8000
Epoch 8/20
5/5 [==============

TypeError: ignored